##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Imágenes en complementos de TensorFlow: operaciones

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/addons/tutorials/image_ops"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/addons/tutorials/image_ops.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/addons/tutorials/image_ops.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
      <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/addons/tutorials/image_ops.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a>
</td>
</table>

## Descripción general

En estas notas se demuestra cómo usar algunas operaciones de imágenes con complementos de TensorFlow.

A continuación, se comparte una lista de las operaciones para imágenes que incluiremos en este ejemplo:

- `tfa.image.mean_filter2d`

- `tfa.image.rotate`

- `tfa.image.transform`

- `tfa.image.random_hsv_in_yiq`

- `tfa.image.adjust_hsv_in_yiq`

- `tfa.image.dense_image_warp`

- `tfa.image.euclidean_dist_transform`

# Preparación

In [ ]:
!pip install -U tensorflow-addons

In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

# Preparación e inspección de las imágenes

## Descargue las imágenes

In [ ]:
img_path = tf.keras.utils.get_file('tensorflow.png','https://tensorflow.org/images/tf_logo.png')

## Inspeccione las imágenes

### Ícono de TensorFlow

In [ ]:
img_raw = tf.io.read_file(img_path)
img = tf.io.decode_image(img_raw)
img = tf.image.convert_image_dtype(img, tf.float32)
img = tf.image.resize(img, [500,500])

plt.title("TensorFlow Logo with shape {}".format(img.shape))
_ = plt.imshow(img)

### Haga una versión en blanco y negro

In [ ]:
bw_img = 1.0 - tf.image.rgb_to_grayscale(img)

plt.title("Mask image with shape {}".format(bw_img.shape))
_ = plt.imshow(bw_img[...,0], cmap='gray')

# Juguemos con tfa.image

## Filtrado medio

El filtrado medio es una técnica de filtrado que, con frecuencia, se usa para despejar el ruido de una imagen o señal. La idea es ir por la imagen, pixel por pixel, y reemplazarlos por los valores promedio de los pixeles que los rodean.

In [ ]:
mean = tfa.image.mean_filter2d(img, filter_shape=11)
_ = plt.imshow(mean)

## Rotación

Con esta operación se rota la imagen dada según el ángulo (en radianes) ingresado por el usuario. 

In [ ]:
rotate = tfa.image.rotate(img, tf.constant(np.pi/8))
_ = plt.imshow(rotate)

## Transformación

Esta operación transforma la imagen dada en base al vector de transformación provisto por el usuario. 

In [ ]:
transform = tfa.image.transform(img, [1.0, 1.0, -250, 0.0, 1.0, 0.0, 0.0, 0.0])
_ = plt.imshow(transform)

## HSV aleatorio en YIQ

Con esta operación se cambia la escala de colores de una imagen RGB dada a YIQ, donde los valores del delta de saturación y matiz se obtienen de forma aleatoria a partir de un rango dado.

In [ ]:
delta = 0.5
lower_saturation = 0.1
upper_saturation = 0.9
lower_value = 0.2
upper_value = 0.8
rand_hsvinyiq = tfa.image.random_hsv_in_yiq(img, delta, lower_saturation, upper_saturation, lower_value, upper_value)
_ = plt.imshow(rand_hsvinyiq)

## Ajuste de HSV en YIQ

Con esta operación se cambia la escala de colores de una imagen RGB dada a YIQ; pero en vez de hacerlo de forma aleatoria, los valores del delta de saturación y matiz son ingresados por el usuario.

In [ ]:
delta = 0.5
saturation = 0.3
value = 0.6
adj_hsvinyiq = tfa.image.adjust_hsv_in_yiq(img, delta, saturation, value)
_ = plt.imshow(adj_hsvinyiq)

## Encapsulamiento de imágenes densas

Esta operación se usa para encapsulamientos (<em>wrap</em>) no lineales de cualquier imagen especificada por el campo de flujo de un vector de compensación (<em>offset</em>) (usado, en este caso, como valores aleatorios). 

In [ ]:
input_img = tf.image.convert_image_dtype(tf.expand_dims(img, 0), tf.dtypes.float32)

flow_shape = [1, input_img.shape[1], input_img.shape[2], 2]
init_flows = np.float32(np.random.normal(size=flow_shape) * 2.0)
dense_img_warp = tfa.image.dense_image_warp(input_img, init_flows)
dense_img_warp = tf.squeeze(dense_img_warp, 0)
_ = plt.imshow(dense_img_warp)

## Transformación de la distancia euclidiana

Con esta operación se actualiza el valor de los pixeles según la distancia euclidiana que hay desde los pixeles en primer plano hasta los del fondo.

- Nota: Para la imagen transformada se toman solamente imágenes y resultados binarios. Si la imagen aportada es diferente, dará como resultado una imagen con un solo valor.

In [ ]:
gray = tf.image.convert_image_dtype(bw_img,tf.uint8)
# The op expects a batch of images, so add a batch dimension
gray = tf.expand_dims(gray, 0)
eucid = tfa.image.euclidean_dist_transform(gray)
eucid = tf.squeeze(eucid, (0, -1))
_ = plt.imshow(eucid, cmap='gray')